In [33]:
import pandas as pd
import numpy as np
import requests
import os
import json
import nflfastpy

pd.options.mode.chained_assignment = None  # default='warn'

IMPORTING SCRAPPED DATA AND PREPROCESSING

In [34]:
#Import and convert json to pd.df
df = pd.read_json("../../../nfl/NFL/nflscraping/espnscores.json")

# Importing additional features from NFLFastPY

nfp_years = [2017, 2018, 2019, 2020, 2021]

#for x in nfp_years:
#    if x == nfp_years[0]:
#        data_nfp = nflfastpy.load_pbp_data(year = x)
#    else:
#        data_nfp = data_nfp.append(nflfastpy.load_pbp_data(year = x)) #loading data

#data_nfp.reset_index(drop=True, inplace=True) 

df_nfl = pd.read_csv("../../../Test/data_nfp.csv", index_col=0)

#Print basic information
print(df.head())
df.describe(include="all")
display(df_nfl)

C:\Users\itrem\AppData\Local\Temp\ipykernel_23024\325231197.py:16: DtypeWarning: Columns (180,181,183,184,190,191,194,195,198,199,204,205,206,207,210,211,214,215,219,220,221,223,225,227,234,235,236,237,238,239,249,250,254,255,256,261,263,264,267,268,269,270,284,285,302) have mixed types. Specify dtype option on import or set low_memory=False.
  df_nfl = pd.read_csv("../../../Test/data_nfp.csv", index_col=0)


   season  week awayteam  hometeam  awayscore  homescore  \
0    2005     1  Raiders  Patriots         20         30   
1    2005     1   Texans     Bills          7         22   
2    2005     1  Bengals    Browns         27         13   
3    2005     1     Jets    Chiefs          7         27   
4    2005     1  Broncos  Dolphins         10         34   

                                           boxscore       idgame  
0  https://espn.com/nfl/boxscore/_/gameId/250908017  [250908017]  
1  https://espn.com/nfl/boxscore/_/gameId/250911002  [250911002]  
2  https://espn.com/nfl/boxscore/_/gameId/250911005  [250911005]  
3  https://espn.com/nfl/boxscore/_/gameId/250911012  [250911012]  
4  https://espn.com/nfl/boxscore/_/gameId/250911015  [250911015]  


,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,out_of_bounds,home_opening_kickoff,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,xpass,pass_oe
0,1,2017_01_ARI_DET,2017091004,DET,ARI,REG,1,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,37,2017_01_ARI_DET,2017091004,DET,ARI,REG,1,ARI,away,DET,...,0,0,-0.287176,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,73,2017_01_ARI_DET,2017091004,DET,ARI,REG,1,ARI,away,DET,...,0,0,0.025196,1.259455,8.791068,7.0,0.652521,0.226989,0.492213,50.778741
3,97,2017_01_ARI_DET,2017091004,DET,ARI,REG,1,ARI,away,DET,...,0,0,-0.362216,NaN,NaN,NaN,NaN,NaN,0.580985,-58.098513
4,118,2017_01_ARI_DET,2017091004,DET,ARI,REG,1,ARI,away,DET,...,0,0,1.880075,0.262409,4.623932,2.0,0.998581,0.998581,0.955220,4.477972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50707,3930,2021_22_LA_CIN,2022021301,CIN,LA,POST,22,CIN,home,LA,...,0,0,-1.242206,NaN,NaN,NaN,NaN,NaN,0.681295,-68.129545
50708,3951,2021_22_LA_CIN,2022021301,CIN,LA,POST,22,NaN,NaN,NaN,...,0,0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50709,3968,2021_22_LA_CIN,2022021301,CIN,LA,POST,22,CIN,home,LA,...,0,0,-0.569208,0.263600,4.354299,2.0,0.998861,0.998861,0.747601,25.239891
50710,3990,2021_22_LA_CIN,2022021301,CIN,LA,POST,22,LA,away,CIN,...,0,0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
# PREPROCESSING DF:

# Filtering data from 2017 on

df = df.loc[df["season"] > 2016]
df.sort_values(by="season", ascending=True, inplace=True)
df.reset_index(drop=True, inplace=True)

# Inserting an absolute score for the game results 

df.insert(df.shape[1],"score_abs",df["homescore"] - df["awayscore"])

# Set winner based on scores, 1 for away team, 0 for away team, 999999999 for draw:

df.insert(df.shape[1],"winner_home", 0)
for i in range(len(df["winner_home"])):
    if df["score_abs"][i] > 0 :
        df["winner_home"][i] = 1
    elif df["score_abs"][i] < 0 :
        df["winner_home"][i] = 0
    else :
        df["winner_home"][i] = 999999999 #Value to filter and remove later on as values are not relevant vs amount of data


df.insert(df.shape[1],"winner_away", 0)
for i in range(len(df["winner_away"])):
    if df["score_abs"][i] < 0 :
        df["winner_away"][i] = 1
    elif df["score_abs"][i] > 0 :
        df["winner_away"][i] = 0
    else :
        df["winner_away"][i] = 999999999 #Value to filter and remove later on as values are not relevant vs amount of data

df.insert(df.shape[1],"winner_team", 0)
for i in range(len(df["winner_team"])):
    if df["score_abs"][i] > 0 :
        df["winner_team"][i] = df["hometeam"][i]
    elif df["score_abs"][i] < 0 :
        df["winner_team"][i] = df["awayteam"][i]
    else :
        df["winner_away"][i] = 999999999 #Value to filter and remove later on as values are not relevant vs amount of dat


# Remove draw result lines as they have no real impact

print("Values before draw games (value 999999999) clean up: \n")
print(df.value_counts("winner_home"))
print(df.value_counts("winner_away"))

drop_lines = df[df["score_abs"] == 0].index
df = df.drop(drop_lines, axis=0)

print("\n Values before draw games (value 999999999) clean up: \n")
print(df.value_counts("winner_home"))
print(df.value_counts("winner_away"))

# Remove boxscore column, no practical information:

df.drop('boxscore', axis=1, inplace=True)


# Remove brackets from idgame, turn field into integer:

df["idgame"] = df["idgame"].apply(lambda x : str(x).strip("[").strip("]").strip("'")).astype(int)


# Redskins changed their name to Washington. Replacing the former with the latter

df["awayteam"].replace("Redskins", "Washington", inplace=True)
df["hometeam"].replace("Redskins", "Washington", inplace=True)

# Reset the index

df.reset_index(drop=True, inplace=True)

# Including a concatenated field that will help us later on include data from a different dataset

df.insert(df.shape[1],"concat", 0)

for i in range(len(df.index)):
    df["concat"][i] = str(df["season"][i])+"_"+str(df["week"][i]).zfill(2)+"_"+df["hometeam"][i]+"_"+df["awayteam"][i]

# Delete useless variables
del drop_lines

df

Values before draw games (value 999999999) clean up: 

winner_home
1            695
0            594
999999999      5
dtype: int64
winner_away
0            695
1            594
999999999      5
dtype: int64

 Values before draw games (value 999999999) clean up: 

winner_home
1    695
0    594
dtype: int64
winner_away
0    695
1    594
dtype: int64


,season,week,awayteam,hometeam,awayscore,homescore,idgame,score_abs,winner_home,winner_away,winner_team,concat
0,2017,9,Bengals,Jaguars,7,23,400951753,16,1,0,Jaguars,2017_09_Jaguars_Bengals
1,2017,4,Bills,Falcons,23,17,400951685,-6,0,1,Bills,2017_04_Falcons_Bills
2,2017,4,Saints,Dolphins,20,0,400950241,-20,0,1,Saints,2017_04_Dolphins_Saints
3,2017,4,Bears,Packers,14,35,400951678,21,1,0,Packers,2017_04_Packers_Bears
4,2017,16,Colts,Ravens,16,23,400951596,7,1,0,Ravens,2017_16_Ravens_Colts
...,...,...,...,...,...,...,...,...,...,...,...,...
1284,2021,5,Lions,Vikings,17,19,401326400,2,1,0,Vikings,2021_05_Vikings_Lions
1285,2021,5,Broncos,Steelers,19,27,401326401,8,1,0,Steelers,2021_05_Steelers_Broncos
1286,2021,5,Dolphins,Buccaneers,17,45,401326402,28,1,0,Buccaneers,2021_05_Buccaneers_Dolphins
1287,2021,15,Cardinals,Lions,12,30,401326544,18,1,0,Lions,2021_15_Lions_Cardinals


In [36]:
#CREATING TEAMS LIST FOR *DF* DATA

#Based on the games played, append both lists "away" and "home" teams even though they both should be the same

df_away = pd.DataFrame(df["awayteam"])
df_away.rename(columns = {"awayteam":'team'}, inplace = True)
df_home = pd.DataFrame(df["hometeam"])
df_home.rename(columns = {"hometeam":'team'}, inplace = True)
df_teams = df_away.append(df_home)

#Sort values
df_teams.sort_values(by="team", inplace = True)

#Drop duplicates
df_teams.drop_duplicates(subset ="team", keep = "first", inplace = True)

#Reset the index
df_teams.reset_index(drop=True, inplace=True)

#Delete useless variables
del df_away
del df_home

#Create working file


#CREATING TEAMS LIST FOR *DF_NFL* DATA

#Based on the games played, append both lists "away" and "home" teams even though they both should be the same

df_nfl_away = pd.DataFrame(df_nfl["away_team"])
df_nfl_away.rename(columns = {"away_team":'team'}, inplace = True)
df_nfl_home = pd.DataFrame(df_nfl["home_team"])
df_nfl_home.rename(columns = {"home_team":'team'}, inplace = True)
df_nfl_teams = df_nfl_away.append(df_nfl_home)

#Sort values
df_nfl_teams.sort_values(by="team", inplace = True)

#Drop duplicates
df_nfl_teams.drop_duplicates(subset ="team", keep = "first", inplace = True)

#Reset the index
df_nfl_teams.reset_index(drop=True, inplace=True)

#Delete useless variables
del df_nfl_away
del df_nfl_home

#Create working file


display(df_teams)
display(df_nfl_teams)

C:\Users\itrem\AppData\Local\Temp\ipykernel_23024\502640664.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_teams = df_away.append(df_home)
C:\Users\itrem\AppData\Local\Temp\ipykernel_23024\502640664.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_nfl_teams = df_nfl_away.append(df_nfl_home)


,team
0,49ers
1,Bears
2,Bengals
3,Bills
4,Broncos
5,Browns
6,Buccaneers
7,Cardinals
8,Chargers
9,Chiefs


,team
0,ARI
1,ATL
2,BAL
3,BUF
4,CAR
5,CHI
6,CIN
7,CLE
8,DAL
9,DEN


PREPROCESSING THE DF THAT WILL CONTAIN DATA FROM LIBRARIE NFLFASTPY, TO ADD NEW FEATURES TO OUR ORIGINAL DF

In [37]:
#Selecting the columns from NFLFastPY that we want to work with

cols = ["season", "week", "season_type", "home_team", "away_team", "home_coach", "away_coach", "location", "weather"] #Selected features that have relevance for our project

#Creating our dataframe and resetting its index
df_nfl = df_nfl[cols]
df_nfl.reset_index(drop=True, inplace=True)

display(df_nfl)



,season,week,season_type,home_team,away_team,home_coach,away_coach,location,weather
0,2017,1,REG,DET,ARI,Jim Caldwell,Bruce Arians,Home,"Controlled Climate Temp: 68° F, Humidity: 70%,..."
1,2017,1,REG,DET,ARI,Jim Caldwell,Bruce Arians,Home,"Controlled Climate Temp: 68° F, Humidity: 70%,..."
2,2017,1,REG,DET,ARI,Jim Caldwell,Bruce Arians,Home,"Controlled Climate Temp: 68° F, Humidity: 70%,..."
3,2017,1,REG,DET,ARI,Jim Caldwell,Bruce Arians,Home,"Controlled Climate Temp: 68° F, Humidity: 70%,..."
4,2017,1,REG,DET,ARI,Jim Caldwell,Bruce Arians,Home,"Controlled Climate Temp: 68° F, Humidity: 70%,..."
...,...,...,...,...,...,...,...,...,...
243126,2021,22,POST,CIN,LA,Zac Taylor,Sean McVay,Neutral,"Clear Temp: 82° F, Humidity: 17%, Wind: WSW 10..."
243127,2021,22,POST,CIN,LA,Zac Taylor,Sean McVay,Neutral,"Clear Temp: 82° F, Humidity: 17%, Wind: WSW 10..."
243128,2021,22,POST,CIN,LA,Zac Taylor,Sean McVay,Neutral,"Clear Temp: 82° F, Humidity: 17%, Wind: WSW 10..."
243129,2021,22,POST,CIN,LA,Zac Taylor,Sean McVay,Neutral,"Clear Temp: 82° F, Humidity: 17%, Wind: WSW 10..."


In [38]:
#Teams have different naming, so we create a dictionary from a csv file to use later on 

nfl_team_dict = dict()
f = open("../05_working_files/team_correspondance_list.csv")
for line in f:
    line = line.strip('\n')
    (key, val) = line.split(";")
    nfl_team_dict[key] = val

print(nfl_team_dict)

{'SF': '49ers', 'CHI': 'Bears', 'CIN': 'Bengals', 'BUF': 'Bills', 'DEN': 'Broncos', 'CLE': 'Browns', 'TB': 'Buccaneers', 'ARI': 'Cardinals', 'LAC': 'Chargers', 'KC': 'Chiefs', 'IND': 'Colts', 'DAL': 'Cowboys', 'MIA': 'Dolphins', 'PHI': 'Eagles', 'ATL': 'Falcons', 'NYG': 'Giants', 'JAX': 'Jaguars', 'NYJ': 'Jets', 'DET': 'Lions', 'GB': 'Packers', 'CAR': 'Panthers', 'NE': 'Patriots', 'LV': 'Raiders', 'LA': 'Rams', 'BAL': 'Ravens', 'NO': 'Saints', 'SEA': 'Seahawks', 'PIT': 'Steelers', 'HOU': 'Texans', 'TEN': 'Titans', 'MIN': 'Vikings', 'WAS': 'Washington'}


In [39]:
#Libraries import contained data from different type of seasons. We only work with Regular (REG), so we get rid of (POST)

drop_lines = df_nfl[df_nfl["season_type"] == "POST"].index
df_nfl = df_nfl.drop(drop_lines, axis=0)

#Index reset after data drop
df_nfl.reset_index(inplace=True)

#We use the dictionary nfl_team_dict we previously created to change team names with the equivalents on our original DF
team_cols = ["home_team", "away_team"]

for g in team_cols:
      for i in range(len(df_nfl[g])):
        df_nfl[g][i] = df_nfl[g][i].replace(str(df_nfl[g][i]), nfl_team_dict[df_nfl[g][i]])

#We create a concatenated field that will allow us to cross filter and add features to our original DF
df_nfl.insert(df_nfl.shape[1]-1,"concat", 0)

for i in range(len(df_nfl.index)):
    df_nfl["concat"][i] = str(df_nfl["season"][i])+"_"+str(df_nfl["week"][i]).zfill(2)+"_"+df_nfl["home_team"][i]+"_"+df_nfl["away_team"][i]

#Grouping by our Concatenated field so we have daily game information only
df_nfl = df_nfl.groupby(by="concat").max()
df_nfl.reset_index(inplace=True)

display(df_nfl)
display(df_nfl.home_team.unique())
display(df_nfl.away_team.unique())

,concat,index,season,week,season_type,home_team,away_team,home_coach,away_coach,location,weather
0,2017_01_49ers_Panthers,703,2017,1,REG,49ers,Panthers,Kyle Shanahan,Ron Rivera,Home,"Clear Temp: 87° F, Humidity: 43%, Wind: Northw..."
1,2017_01_Bears_Falcons,368,2017,1,REG,Bears,Falcons,John Fox,Dan Quinn,Home,"Sunny Temp: 64° F, Humidity: 57%, Wind: SE 10 mph"
2,2017_01_Bengals_Ravens,538,2017,1,REG,Bengals,Ravens,Marvin Lewis,John Harbaugh,Home,"Sunny Temp: 68° F, Humidity: 43%, Wind: SE 9 mph"
3,2017_01_Bills_Jets,1941,2017,1,REG,Bills,Jets,Sean McDermott,Todd Bowles,Home,"Sun & clouds Temp: 65° F, Humidity: 53%, Wind:..."
4,2017_01_Broncos_Chargers,1425,2017,1,REG,Broncos,Chargers,Vance Joseph,Anthony Lynn,Home,"Mostly Cloudy Temp: 73° F, Humidity: 39%, Wind..."
...,...,...,...,...,...,...,...,...,...,...,...
1291,2021_18_Raiders_Chargers,239409,2021,18,REG,Raiders,Chargers,Rich Bisaccia,Brandon Staley,Home,"Clear Temp: 59° F, Humidity: 20%, Wind: NE 10 mph"
1292,2021_18_Rams_49ers,240502,2021,18,REG,Rams,49ers,Sean McVay,Kyle Shanahan,Home,"Sunny Temp: 74° F, Humidity: 20%, Wind: W 2 mph"
1293,2021_18_Ravens_Steelers,240131,2021,18,REG,Ravens,Steelers,John Harbaugh,Mike Tomlin,Home,"Cloudy, rain Temp: 38° F, Humidity: 90%, Wind:..."
1294,2021_18_Texans_Titans,240671,2021,18,REG,Texans,Titans,David Culley,Mike Vrabel,Home,"N/A Temp: 73° F, Humidity: 85%, Wind: WSW 7 mph"


array(['49ers', 'Bears', 'Bengals', 'Bills', 'Broncos', 'Browns',
       'Cowboys', 'Lions', 'Packers', 'Patriots', 'Rams', 'Texans',
       'Titans', 'Vikings', 'Washington', 'Buccaneers', 'Chargers',
       'Chiefs', 'Colts', 'Falcons', 'Giants', 'Jaguars', 'Panthers',
       'Raiders', 'Ravens', 'Saints', 'Seahawks', 'Steelers', 'Cardinals',
       'Eagles', 'Jets', 'Dolphins'], dtype=object)

array(['Panthers', 'Falcons', 'Ravens', 'Jets', 'Chargers', 'Steelers',
       'Giants', 'Cardinals', 'Seahawks', 'Chiefs', 'Colts', 'Jaguars',
       'Raiders', 'Saints', 'Eagles', 'Texans', 'Cowboys', 'Bears',
       'Dolphins', 'Packers', 'Lions', 'Titans', 'Bills', 'Washington',
       'Browns', 'Patriots', '49ers', 'Vikings', 'Rams', 'Broncos',
       'Bengals', 'Buccaneers'], dtype=object)

PREPROCESSING OF THE DF_NFL DATA CONTAINING THE NEW FEATURES

In [40]:
# Most of the new features are located in the Weather column. Starting by setting it to lower caps

df_nfl['weather'] = df_nfl['weather'].str.lower()

# After analyzing the data present, we create a dictionary with all the generic terms that we want to remove

terms2remove = {'temp': '', 
                '81 degrees': '', 
                'n/a': '', 
                '(': '',
                ')': '',
                'warm': '',
                'mostly': '',
                'partly': '',
                'coudy': 'cloudy',
                'with periods of rain': '',
                'likely': '',
                'warm': '',
                'mid 60s and ': '',
                '30% chance of': '',
                'showers': 'rain',
                'indoors': 'indoor',
                'skies': '',
                'heavy lake effect': '',
                '&': 'and',
                'clear': 'sunny',
                'indoor': 'indoor',
                'chance 40%': '',
                'clear': '',
                'clouidy': 'cloudy',
                'rainy': 'rain',
                'windy': 'wind',
                'cloundy': 'cloudy',
                'controlled climate': 'indoor',
                'hazy': 'cloudy',
                'cold': '',                             
                'scattered': '',
                'cool': '', 
                'warm': '',
                'overcast': 'cloudy',
                'shower': '',
                'mild': '', 
                'breezy': 'sunny',
                'cloudy with rain and wind': 'rain',
                'raining': 'rain', 
                'rain and wind': 'rain',
                'freezing': '', 
                'foggy': 'fog',
                'cloudy with rain': 'rain',
                'light snow/fog': 'fog',
                'cloudy/wind': 'cloudy',
                'cloudy and wind': 'cloudy',
                'controlled climage': 'indoor',
                'cloudy with snow flurries': 'cloudy',
                'sunny with some clouds': 'cloudy', 
                'clouds': 'cloudy',
                'hazy': 'cloudy',
                'brisk': '', 
                'cloudy with wind': 'cloudy', 
                'cloudly': 'cloudy',
                'blustery': '',
                'unseasonably': '',
                'snow rain/wind':'snow',
                'nan': 'cloudy',              
                'sun and clouds': 'sunny',    
                'dry': 'sunny',                  
                'outdoor': 'sunny',
                'party': '',
                'light': '',
                'and': '',
                'suncloudy': 'sunny',
                ' ' : ''
                }

terms2remove_keys = list(terms2remove.keys())

for i in range(len(df_nfl["weather"])):
    for x in terms2remove_keys:
        df_nfl["weather"][i] = str(df_nfl["weather"][i]).replace(str(x),terms2remove[x])


# We split Weather into different columns
df_nfl_weather = df_nfl["weather"].str.split(",",expand=True)


display(df_nfl)
display(df_nfl_weather)




,concat,index,season,week,season_type,home_team,away_team,home_coach,away_coach,location,weather
0,2017_01_49ers_Panthers,703,2017,1,REG,49ers,Panthers,Kyle Shanahan,Ron Rivera,Home,":87°f,humidity:43%,wind:northwest8mph"
1,2017_01_Bears_Falcons,368,2017,1,REG,Bears,Falcons,John Fox,Dan Quinn,Home,"sunny:64°f,humidity:57%,wind:se10mph"
2,2017_01_Bengals_Ravens,538,2017,1,REG,Bengals,Ravens,Marvin Lewis,John Harbaugh,Home,"sunny:68°f,humidity:43%,wind:se9mph"
3,2017_01_Bills_Jets,1941,2017,1,REG,Bills,Jets,Sean McDermott,Todd Bowles,Home,"suncloudy:65°f,humidity:53%,wind:nne6mph"
4,2017_01_Broncos_Chargers,1425,2017,1,REG,Broncos,Chargers,Vance Joseph,Anthony Lynn,Home,"cloudy:73°f,humidity:39%,wind:southwest12mph"
...,...,...,...,...,...,...,...,...,...,...,...
1291,2021_18_Raiders_Chargers,239409,2021,18,REG,Raiders,Chargers,Rich Bisaccia,Brandon Staley,Home,":59°f,humidity:20%,wind:ne10mph"
1292,2021_18_Rams_49ers,240502,2021,18,REG,Rams,49ers,Sean McVay,Kyle Shanahan,Home,"sunny:74°f,humidity:20%,wind:w2mph"
1293,2021_18_Ravens_Steelers,240131,2021,18,REG,Ravens,Steelers,John Harbaugh,Mike Tomlin,Home,"cloudy,rain:38°f,humidity:90%,wind:sw3mph"
1294,2021_18_Texans_Titans,240671,2021,18,REG,Texans,Titans,David Culley,Mike Vrabel,Home,":73°f,humidity:85%,wind:wsw7mph"


,0,1,2,3,4
0,:87°f,humidity:43%,wind:northwest8mph,None,None
1,sunny:64°f,humidity:57%,wind:se10mph,None,None
2,sunny:68°f,humidity:43%,wind:se9mph,None,None
3,suncloudy:65°f,humidity:53%,wind:nne6mph,None,None
4,cloudy:73°f,humidity:39%,wind:southwest12mph,None,None
...,...,...,...,...,...
1291,:59°f,humidity:20%,wind:ne10mph,None,None
1292,sunny:74°f,humidity:20%,wind:w2mph,None,None
1293,cloudy,rain:38°f,humidity:90%,wind:sw3mph,None
1294,:73°f,humidity:85%,wind:wsw7mph,None,None


In [41]:
# Based on the data in df_nfl_weather, we rename and append the columns to our DF_NFL dataframe, starting by the temperature

name = "temp"
df_nfl_weather2 = df_nfl_weather[0].str.split(":",expand=True)
for i in range(df_nfl_weather.shape[1]):
    df_nfl_weather2.rename(columns = {i:name+str(i)}, inplace=True)

terms2remove = {'humidity': '',
                ';h': '',
                '0°f': '',
                '°f': '',
                ' ':''
                }

terms2remove_keys = list(terms2remove.keys())

for i in range(len(df_nfl_weather2["temp1"])):
    for x in terms2remove_keys:
        df_nfl_weather2["temp1"][i] = str(df_nfl_weather2["temp1"][i]).replace(str(x),terms2remove[x])

display(df_nfl_weather2)
df_nfl = df_nfl.join(df_nfl_weather2)

# Renaming and appending Humidity columns
name = "humidity"
df_nfl_weather2 = df_nfl_weather[1].str.split(":",expand=True)
for i in range(df_nfl_weather.shape[1]):
    df_nfl_weather2.rename(columns = {i:name+str(i)}, inplace=True)


terms2remove = {'89°f': '',
                '°f': '',
                '39°f': '',
                '56°f': '',
                '61°f' : '',
                'nw5mph' : '',
                '24°f' : '',
                'nw12mph' : '',
                '55°f' : '',
                'southwest13mphmph' : '',
                '43°f' : '',
                '40°f' : '',
                '46°f' : '',
                '65°f' : '',
                'wind' : '',
                '79°f' : '',
                'southwest10mph' : '',
                '52°f' : '',
                '69°f' : '',
                '38°f' : '',
                '45°f' : '',
                '57°f' : '',
                '81°f' : '',
                '67°f' : '',
                '64°f' : '',
                '68°f' : '',
                'mph': '',
                '%':'',
                ' ':''                
                }

terms2remove_keys = list(terms2remove.keys())

for i in range(len(df_nfl_weather2["humidity1"])):
    for x in terms2remove_keys:
        df_nfl_weather2["humidity1"][i] = str(df_nfl_weather2["humidity1"][i]).replace(str(x),terms2remove[x])

display(df_nfl_weather2)
df_nfl = df_nfl.join(df_nfl_weather2)

# Renaming and appending Wind columns
name = "wind"
df_nfl_weather2 = df_nfl_weather[2].str.split(":",expand=True)
for i in range(df_nfl_weather.shape[1]):
    df_nfl_weather2.rename(columns = {i:name+str(i)}, inplace=True)


terms2remove = {'north': '',
                'south':'',
                'east': '',
                'west': '',
                'n': '',
                's': '',            
                'e': '',
                'w': '',
                'no':'',
                '14-':'',
                '12-':'',
                '15gutupto':'',
                '14gutigto': '',
                '/':'',
                'rly':'',
                'by':'',
                '10-':'',
                'dg f': '',
                'calm':'',
                'from11-':'',
                'from': '',
                'o': '',
                'N':'',
                '-': '',
                'mph':'',
                '48%' : '',
                '70°f' : '',
                '88%' : '',
                '89%' : '',
                '44%' : '',
                '83%' : '',
                '77%' : '',
                '45%' : '',
                '97%' : '',
                '91%' : '',
                '31%' : '',
                '54%' : '',
                '41%' : '',
                '56%' : '',
                '34dgf' : '',
                '27%' : '',
                '49%' : '',
                '93%' : '',
                '47%' : '',
                '90%' : '',
                ' ':''                
                }

terms2remove_keys = list(terms2remove.keys())

for i in range(len(df_nfl_weather2["wind1"])):
    for x in terms2remove_keys:
        df_nfl_weather2["wind1"][i] = str(df_nfl_weather2["wind1"][i]).replace(str(x),terms2remove[x])


display(df_nfl_weather2)
df_nfl = df_nfl.join(df_nfl_weather2)


#Removing useless columns
df_nfl.drop(['temp2','temp3', 'temp4', 'humidity2', 'humidity0', 'wind0' ], axis=1, inplace=True)

df_nfl.rename(columns = {"temp0":'weather_type', "temp1": "temperature", "humidity1":'humidity',"wind1":'wind' }, inplace = True)

#Removing some additional useless data
terms2remove = {'suncloudy': 'cloudy',
                't' : ''                
                }

terms2remove_keys = list(terms2remove.keys())

for i in range(len(df_nfl["weather_type"])):
    for x in terms2remove_keys:
        df_nfl["weather_type"][i] = str(df_nfl["weather_type"][i]).replace(str(x),terms2remove[x])

#Configuring the right type of column

#cols = ["temperature", "humidity", "wind"]

#for x in cols:
#    df_nfl[x] = df_nfl[x].astype("int64")

# Preparing to complete all empty fields, we select the columns to work on

col_nan = ['weather_type', 'temperature', 'humidity', 'wind']

#Then we change all missing values by nan
for x in col_nan:
    for i in range(len(df_nfl[x])):
        if df_nfl[x][i] == "" or df_nfl[x][i] == "None":
           df_nfl[x][i] = np.nan
        else:
            pass

#We differenciate the columns based on their type to apply the most appropriate method
cateogry_columns=df_nfl.select_dtypes(include=['object','string']).columns.tolist()
integer_columns=df_nfl.select_dtypes(include=['int32','int64','float64']).columns.tolist()

for column in df_nfl:
    if df_nfl[column].isnull().any():
        if(column in cateogry_columns):
            df_nfl[column]=df_nfl[column].fillna(df_nfl[column].mode()[0])
        else:
            df_nfl[column]=df_nfl[column].fillna(df_nfl[column].mean)

#Dropping some columns with information not usable anymore
cols_drop = ["index", "season_type", "weather"]

for x in cols_drop:
    df_nfl.drop(x, axis=1, inplace=True)



,temp0,temp1,temp2,temp3,temp4
0,,87,None,None,None
1,sunny,64,None,None,None
2,sunny,68,None,None,None
3,suncloudy,65,None,None,None
4,cloudy,73,None,None,None
...,...,...,...,...,...
1291,,59,None,None,None
1292,sunny,74,None,None,None
1293,cloudy,None,None,None,None
1294,,73,None,None,None


,humidity0,humidity1,humidity2
0,humidity,43,None
1,humidity,57,None
2,humidity,43,None
3,humidity,53,None
4,humidity,39,None
...,...,...,...
1291,humidity,20,None
1292,humidity,20,None
1293,rain,38,None
1294,humidity,85,None


,wind0,wind1
0,wind,8
1,wind,10
2,wind,9
3,wind,6
4,wind,12
...,...,...
1291,wind,10
1292,wind,2
1293,humidity,
1294,wind,7


ADDING FEATURES FROM *DF_NFL* TO *DF*

In [42]:
#Selecting the features that will provide data

cols_merge = ['home_coach', 'away_coach', 'location', 'weather_type', 'temperature', 'humidity', 'wind']

#Creating the columns in DF
for x in cols_merge:
    df.insert(df.shape[1], x, "") 

#Adding the data to DF 
for z in df["concat"]:
    if df_nfl.index[df_nfl["concat"] == z].tolist() != []:
        a = df_nfl.index[df_nfl["concat"]== z].tolist()
        b = df.index[df["concat"] == z].tolist()
        for x in cols_merge:
            df[x][b[0]] = df_nfl[x][a[0]]
    else:
        c = df.index[df["concat"] == z].tolist()
        for x in cols_merge:
            df[x][c[0]] = np.nan

df.isnull().any()
display(df)

df.to_csv('../04_datasets/nfl_dataset.csv')  


,season,week,awayteam,hometeam,awayscore,homescore,idgame,score_abs,winner_home,winner_away,winner_team,concat,home_coach,away_coach,location,weather_type,temperature,humidity,wind
0,2017,9,Bengals,Jaguars,7,23,400951753,16,1,0,Jaguars,2017_09_Jaguars_Bengals,Doug Marrone,Marvin Lewis,Home,cloudy,78,74,12
1,2017,4,Bills,Falcons,23,17,400951685,-6,0,1,Bills,2017_04_Falcons_Bills,Dan Quinn,Sean McDermott,Home,cloudy,68,70,5
2,2017,4,Saints,Dolphins,20,0,400950241,-20,0,1,Saints,2017_04_Dolphins_Saints,Adam Gase,Sean Payton,Neutral,cloudy,63,64,13
3,2017,4,Bears,Packers,14,35,400951678,21,1,0,Packers,2017_04_Packers_Bears,Mike McCarthy,John Fox,Home,cloudy,65,62,5
4,2017,16,Colts,Ravens,16,23,400951596,7,1,0,Ravens,2017_16_Ravens_Colts,John Harbaugh,Chuck Pagano,Home,cloudy,63,78,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1284,2021,5,Lions,Vikings,17,19,401326400,2,1,0,Vikings,2021_05_Vikings_Lions,Mike Zimmer,Dan Campbell,Home,indoor,66,69,12
1285,2021,5,Broncos,Steelers,19,27,401326401,8,1,0,Steelers,2021_05_Steelers_Broncos,Mike Tomlin,Vic Fangio,Home,sunny,73,63,8
1286,2021,5,Dolphins,Buccaneers,17,45,401326402,28,1,0,Buccaneers,2021_05_Buccaneers_Dolphins,Bruce Arians,Brian Flores,Home,cloudy,84,60,6
1287,2021,15,Cardinals,Lions,12,30,401326544,18,1,0,Lions,2021_15_Lions_Cardinals,Dan Campbell,Kliff Kingsbury,Home,indoor,68,70,5


In [43]:
df.isnull().any()

season          False
week            False
awayteam        False
hometeam        False
awayscore       False
homescore       False
idgame          False
score_abs       False
winner_home     False
winner_away     False
winner_team     False
concat          False
home_coach      False
away_coach      False
location        False
weather_type    False
temperature     False
humidity        False
wind            False
dtype: bool

In [21]:
df_nfl.humidity = df_nfl.humidity.asty

In [22]:
df_nfl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296 entries, 0 to 1295
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   concat        1296 non-null   object
 1   season        1296 non-null   int64 
 2   week          1296 non-null   int64 
 3   home_team     1296 non-null   object
 4   away_team     1296 non-null   object
 5   home_coach    1296 non-null   object
 6   away_coach    1296 non-null   object
 7   location      1296 non-null   object
 8   weather_type  1296 non-null   object
 9   temperature   1296 non-null   int64 
 10  humidity      1296 non-null   string
 11  wind          1296 non-null   object
dtypes: int64(3), object(8), string(1)
memory usage: 121.6+ KB
